In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import time

In [ ]:
IMAGE_SIZE = 224

In [ ]:
!cp ../input/happy-whale-and-dolphin/sample_submission.csv sample_submission.csv
!cp ../input/happy-whale-and-dolphin/train.csv train.csv
!mkdir test_images
!mkdir train_images

In [ ]:
train = pd.read_csv("./train.csv")
train.index = train["image"].copy()
train["image"] = train["image"].str[:-3] + "bmp"
train["bbox"] = pd.read_csv("../input/whale2-cropped-dataset/train2.csv", index_col="image")["box"].fillna("").map(lambda x: list(map(int, x.split(" "))) if x!="" else [])
train

In [ ]:
sample_submission = pd.read_csv("./sample_submission.csv")
sample_submission.index = sample_submission["image"].copy()
sample_submission["inference_image"] = sample_submission["image"].str[:-3] + "bmp"
sample_submission["bbox"] = pd.read_csv("../input/whale2-cropped-dataset/test2.csv", index_col="image")["box"].fillna("").map(lambda x: list(map(int, x.split(" "))) if x!="" else [])
sample_submission

In [ ]:
train.to_csv("./train.csv", index=False)
sample_submission.to_csv("./sample_submission.csv", index=False)

In [ ]:
def copy_dir(dirname, base_path="../input/happy-whale-and-dolphin/"):
    bboxes = train["bbox"] if dirname=="train_images" else sample_submission["bbox"]
    print("Copying", dirname)
    path = os.path.join(base_path, dirname)
    images = list(os.listdir(path))
    n = len(images)
    for i, f in enumerate(images):
        print(f"{i}/{n}", end="\r")
        image_path = os.path.join(path, f)
        image = cv2.imread(image_path)
        if len(bboxes[f]):
            bbox = bboxes[f]
            xmin, ymin, xmax, ymax = bbox
            image = image[ymin:ymax, xmin:xmax] # crop image
        image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE), interpolation=cv2.INTER_CUBIC)
        new_path = os.path.join("./", dirname, f.split('.')[0] + ".bmp")
        cv2.imwrite(new_path, image)
    
copy_dir("train_images")
copy_dir("test_images")